In [1]:
from probemapper.registration import ANTSPointTransformation, ANTSTransformationStages
from probemapper.io import save_nifti
import os, subprocess
import numpy as np
import pandas as pd

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220526_P189_laqueus_DiI_L-J30/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, f"indexing")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Transform channel coordinates

In [4]:
T = ANTSPointTransformation(
    ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin/",
    num_threads=8
)
T.set_transformations([
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_0GenericAffine.mat"), inverse=True),
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_1InverseWarp.nii.gz"), inverse=False),
])
T.set_moving_points(os.path.join(rootdir, "analysis/skeleton/channel_skeleton.csv"))
T.set_outname(os.path.join(tmpdir, "channel_LUT.csv"))

In [5]:
T.run()

['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsApplyTransformsToPoints', '-d', '3', '-i', '/bucket/ReiterU/DBS/data/20220526_P189_laqueus_DiI_L-J30/analysis/skeleton/channel_skeleton.csv', '-o', '/flash/ReiterU/tmp/P189/indexing/channel_LUT.csv', '-t', '[/bucket/ReiterU/DBS/data/20220526_P189_laqueus_DiI_L-J30/analysis/ants/F2M_0GenericAffine.mat,1]', '-t', '/bucket/ReiterU/DBS/data/20220526_P189_laqueus_DiI_L-J30/analysis/ants/F2M_1InverseWarp.nii.gz']



### Query region ID

In [6]:
from probemapper import skeletonization

In [7]:
df = pd.read_csv(os.path.join(tmpdir, "channel_LUT.csv"))
coords = np.array([df["Z"], df["Y"], df["X"]])

In [8]:
regions = skeletonization.query_region_id2(coords,
                     "/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                     100, coords.shape[1]-100, 10)

In [9]:
df["Region"] = regions

In [10]:
df.insert(0, "channel", df.pop("channel"))

In [11]:
display(df)

,channel,X,Y,Z,X_org,Y_org,Z_org,Region
0,-100,1527.27,2421.650,815.316,1506.84,3033.210,910.192,3
1,-99,1530.92,2414.380,823.537,1509.64,3025.230,914.787,3
2,-98,1534.56,2407.080,831.787,1512.45,3017.240,919.383,3
3,-97,1538.20,2399.750,840.067,1515.26,3009.260,923.978,3
4,-96,1541.82,2392.380,848.380,1518.07,3001.280,928.573,3
...,...,...,...,...,...,...,...,...
347,247,2294.35,951.369,3538.800,1971.29,357.306,2887.060,1
348,248,2293.72,948.311,3548.100,1970.29,351.514,2895.240,1
349,249,2292.82,945.753,3557.490,1969.10,345.816,2903.480,1
350,250,2291.59,943.695,3567.000,1967.73,340.212,2911.780,1


In [12]:
df.to_csv(os.path.join(tmpdir, "channel_LUT.csv"), index=False)

In [13]:
sk = skeletonization.generate_skeleton_image(coords[:, 100:],
                             "/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                             10)

In [14]:
sk = sk.astype("uint8")
save_nifti(sk, os.path.join(tmpdir, "probe_image.nii.gz"), 1, 1, 1)

In [15]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/P189/indexing', 'deigo:/bucket/ReiterU/DBS/data/20220526_P189_laqueus_DiI_L-J30/analysis'], returncode=0)